# Iskanje kvazi-trkov SHA-1 
Generiram naključen niz, pogledam njegov hash. Preverim, ce se je zacetek tega hasha ze kdaj pojavil pri katerem izmed prej generiranih hashov. 
- Če se je pojavil zakljucim in za moji besedili vzamem trenutno generirano besedilo in besedilo, ko je dalo isti začetek hasha.
- Če se ni pojavil besedilo shranim v slovar pod ključem začetka hasha. 

In [ ]:
import string
import random 
import hashlib

# pomožne funkcije
def random_string(min_len=10, max_len=15):
    return ''.join(random.choices(string.ascii_uppercase + string.digits, k = random.randint(min_len, max_len)))    

def sha1(s):
    return hashlib.sha1(s.encode()).hexdigest()

In [ ]:
hashes = {}
while True:
    s1 = random_string()
    if(sha1(s1)[:11] in hashes):
        break
    else:
        hashes[sha1(s1)[:11]] = s1
s2 = hashes[sha1(s1)[:11]]

print(sha1(s1),s1)
print(sha1(s2),s2)


## Primer najdenih besedil 
shranjeni z razlogom krajšanja časa izvajanja

In [ ]:
s1 = "5V38DYZKVPCX5W"
s2 = "0CI603K1K2"

# Podpis rešitve

priprava besedila za podpis:

In [ ]:
s = f"{s1} {s2}"
text_sha = sha1(s)
text_int = int(sha1(s), 16)
first_solution = f"{s} {text_int}"
print(first_solution)

## Generiranje ključa 
`pip install PyCryptodome`  

Ključ generiram s pomočjo knjižnice in izvozim potrebne vrednosti

In [ ]:
from Crypto.PublicKey import DSA
key = DSA.generate(1024)
q = key.q
p = key.p
a = key.x
alpha = key.g
beta = key.y
print(f"{p}\n{q}\n{alpha}\n{beta}")
print(f"a:{a}")

### shranjena identiteta

In [ ]:
identity = "63180016JanezJustin"
a = 88769319014054660730681526478685395170035564236
p = 150062830636387325240554145885519272280749208800939775265138904737897223975352708831313987848753708078363982052997305312195973671863429081630615348449025232191175751359619885970801765526680019929819779817734763444389761704863824924813191029707940053041375422807192070838830680716520059121293732085522168600171
q = 755819435904388790217062696519171866591972627633
alpha = 88629923388766247370427034792874653217028414506418334706536728355400045695704287033788704130284124131962016875579895492109240843123396552403700123722906619600116179463917448376716958697224244414083771453611764634025044515036449070553801652648528923205740422980648372351861494147732463178840005934326607498429
beta = 122959898742959017778098446549445363233475078032748271993764366858345385956191321272931294528184550565008338503358550376275689421821009799962775160088021762402931749706368041723858842832367606808757213183773599056202196552162236729798594970563347932259735582568296967906840243355450986983758401243265415216356
full_identity = f"{identity}\n{p}\n{q}\n{alpha}\n{beta}"
print(full_identity)

## Podpisovanje
Se izvede po algoritmu opisanem na prosojnicah. Podpis se po kreiranju tudi preveri

In [ ]:
from random import randint
while True:
    k = randint(1, q-1)
    gamma = pow(alpha, k, p) % q
    k_inv = pow(k, -1, q)
    delta = (k_inv * (text_int + a * gamma)) % q
    if delta != 0 and gamma != 0:
        break
print(gamma, delta)

# preverba podpisa
w = pow(delta, -1, q)
e1 = (text_int*w) % q
e2 = (gamma*w) % q
v = ((pow(alpha,e1,p) * pow(beta,e2,p)) % p) % q

if v==gamma:
    print("Signature ok")
else:
    print("Signature invalid")

# Blockchain
## Generiranje novega bloka
Poskušam z različnimi vrednostim 4. vrstice, dokler ne dobim bloka, ki ima na začetku hasha dovolj ničel

In [ ]:
prev_block_hash = "00000000679401e5c44f03f801ffe47ce359c59c"
my_block_3 = f"{first_solution}\n{identity} {gamma} {delta}\n{prev_block_hash}\n"
n = 7
i = 0
prefix = n*"0"
while True:
    my_block = my_block_3 + str(i)
    if(sha1(my_block)[:n] == prefix):
        break
    i+=1
my_block = f"{my_block}\n{sha1(my_block)}"
print(my_block)

## Preverjanje bloka
Preveri, če je blok pravilen ob predpostavki, da je blok pred njim pravilen 

In [ ]:
last_block = """
5V38DYZKVPCX5W 0CI603K1K2 479715696515895985103007358854059603148610070068
63180016JanezJustin 296674042762533888762414411018612906390837867511 435220522721221093656746386677426046316853205572
00000000679401e5c44f03f801ffe47ce359c59c
412351070
00000001c32b849ad42355b220ed81f4eacaec0d
""".split("\n")
last_block_id = """
150062830636387325240554145885519272280749208800939775265138904737897223975352708831313987848753708078363982052997305312195973671863429081630615348449025232191175751359619885970801765526680019929819779817734763444389761704863824924813191029707940053041375422807192070838830680716520059121293732085522168600171
755819435904388790217062696519171866591972627633 
88629923388766247370427034792874653217028414506418334706536728355400045695704287033788704130284124131962016875579895492109240843123396552403700123722906619600116179463917448376716958697224244414083771453611764634025044515036449070553801652648528923205740422980648372351861494147732463178840005934326607498429
122959898742959017778098446549445363233475078032748271993764366858345385956191321272931294528184550565008338503358550376275689421821009799962775160088021762402931749706368041723858842832367606808757213183773599056202196552162236729798594970563347932259735582568296967906840243355450986983758401243265415216356
""".split("\n")
prev_block_hash = "00000000679401e5c44f03f801ffe47ce359c59c"
text_arr = last_block[1].split(" ")
text_int2 = int(text_arr[2])

In [ ]:
### Preveri, če se podani hashi ujemajo
# v trenutnem bloku je pravilen hash prejšnjega bloka
isOk = True
if prev_block_hash != last_block[3]:
    print("Hash not matching previous block")
    isOk = False
# hash trenutnega bloka je pravilen
if last_block[5] != sha1('\n'.join(last_block[1:5])):
    print("Hash of current block is invalid")
    isOk = False

### preveri, če je besedilo pravilno
# preveri, če so prefixi hashov pravilni
if sha1(text_arr[0])[:11] != sha1(text_arr[1])[:11]: 
    print("line 1 hash prefixes not matching")
    isOk = False
# preveri, če je int zapis hasha pravilen
if str(text_int2) != text_arr[2]:
    print("Int representation of hash is invalid")
    isOk = False
if isOk:
    print("Hashes ok")

In [ ]:
### Preveri podpis
# podpis
[g, d] = last_block[2].split(" ")[1:3]
g = int(g)
d = int(d)

# izluscevanje identitete
p = int(last_block_id[1])
q = int(last_block_id[2])
al = int(last_block_id[3])
b = int(last_block_id[4])

# preverba podpisa po algoritmu iz predavanj
w = pow(d, -1, q)
e1 = (text_int2*w) % q
e2 = (g*w) % q
v = ((pow(al,e1,p) * pow(b,e2,p)) % p) % q

if v==g:
    print("Signature ok")
else:
    print("Signature invalid")